<a href="https://colab.research.google.com/github/ClauCll/Curso_ML_A_Z/blob/master/Regresi%C3%B3n/Regresion_Bajas_Antiguedad_v8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cuantas serán las bajas voluntarias de los clientes fijos para un periodo x. (Agosto - Diciembre 2020)
 

In [295]:
import pandas as pd
import numpy as np

#Gráfico
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt 
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns

import gspread
from oauth2client.client import GoogleCredentials 
from google.colab import auth
# Autoriza Claudia
auth.authenticate_user()
gc = gspread.authorize (GoogleCredentials.get_application_default())

### leemos datos

In [296]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [297]:
data = pd.read_csv('/content/drive/My Drive/Dataset/M_BAJAS.txt',delimiter='~')
data.Fecha = pd.to_datetime(data.FECHA_BAJA)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [ ]:
data.head(5)

In [298]:
encabezados= ['Año','Mes','Fecha','Antiguedad','Cant_Bajas']
data.columns=encabezados

In [ ]:
data.head(5)

In [ ]:
 #validamos datos faltantes
 print(pd.isnull(data).sum())

In [ ]:
data.describe(include='all')

In [ ]:
data.info 

In [300]:
#Agrupamos la columna antiguedad
#1 : 1 TRIMESTRE (0-93 DIAS)
#2 : 2 TRIMESTRE (94-186 DIAS)
#3 : 3 TRIMESTRE (187-279 DIAS)
#4 : 4 TRIMESTRE (280-370 DIAS)
#5 : A 2 AÑOS (371-725 DIAS) 
#6 : A 3 AÑOS (726-1095 DIAS)
#7 : 4 años en adelante (Máximo DIAS 7276) 
bins=[-1,93,186,279,370,725,1095,7276]
names=['1','2','3','4','5','6','7']
data['Antiguedad'] = pd.cut(data['Antiguedad'],bins,labels=names)

In [301]:
#validamos datos sin clasificar
data[pd.isnull(data.Antiguedad)]

,Año,Mes,Fecha,Antiguedad,Cant_Bajas


In [302]:
#pasamos datos a clasificados a entero
data['Antiguedad'] = data['Antiguedad'].astype('int64')

In [ ]:
data.dtypes

In [ ]:
data.head(5)

In [ ]:
 #validamos datos faltantes
 print(pd.isnull(data).sum())

In [304]:
grupo = data.groupby(by = ['Año','Mes','Antiguedad']).sum()
grupo= pd.DataFrame(grupo).reset_index()

In [ ]:
grupo.head(5)

In [ ]:
 #validamos datos faltantes
 print(pd.isnull(grupo).sum())

In [96]:
#grupo.dropna(subset = ['Cant_Bajas'] , axis=0, inplace=True)

In [305]:
x= grupo.iloc[:,:-1].values
x=np.array(x)
y= grupo.iloc[:,3].values
y = y[:,np.newaxis]

In [306]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

labelencoder= LabelEncoder()
x[:,2] = labelencoder.fit_transform(x[:,2])
ct = ColumnTransformer([('one_hot_encoder',OneHotEncoder(categories='auto'),[2])],remainder='passthrough')
x=np.array(ct.fit_transform(x),dtype=np.int)

In [307]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=0)

In [308]:
from sklearn.preprocessing import PolynomialFeatures
poly_reg = PolynomialFeatures(degree = 3)
x_poly_train = poly_reg.fit_transform(x_train)
x_poly_test = poly_reg.fit_transform(x_test)

from sklearn.linear_model import LinearRegression
lineal_poly = LinearRegression()
lineal_poly.fit(x_poly_train,y_train)

print("precisión del modelo regresión polinomial")
lineal_poly.score(x_poly_train,y_train)

precisión del modelo regresión polinomial


0.881223363134311

In [309]:
y_predic = lineal_poly.predict(x_poly_test)

In [ ]:
#Gráfica
fig, ax = plt.subplots()
for ciclo in list(range (2019,2021)):
  print(ciclo)
  if ciclo == 2019: 
    print ('sombrero')
    marca ="^"
  else:
    print ('cero')
    marca ="o"
  ax.scatter(
    x_train[0,0],
    x_mes[ c0[:,0]==1,0],
    Y[ c0[:,0]==1,0],
    c="black",
    marker = marca,
    label = ciclo,
    alpha=0.5)
plt.legend()
plt.show()

In [ ]:
pd.DataFrame(x_test)

In [ ]:
pd.DataFrame(y_test)

In [243]:
x_train[ x_train[:,5]==1,8]

array([ 5, 10,  8,  9,  6,  1,  6, 11,  7,  3, 12,  2,  3,  5,  7])

In [ ]:
y_train[ y_train[:,5]==1,8]

In [325]:
cont_reg=0
acum1=0
acum2=0
acum3=0
acum4=0
acum5=0
acum6=0
acum7=0
acum8=0
acum9=0
acum10=0
acum11=0
acum12=0
for _ in x_train[:,8]:
  if x_train[cont_reg,7] == 2020:
    if x_train[cont_reg,8]== 1:
      acum1=acum1+ y_train[cont_reg]
    else:
      if x_train[cont_reg,8]== 2:
        acum2=acum2+ y_train[cont_reg]
      else:
        if x_train[cont_reg,8]== 3:
          acum3=acum3+ y_train[cont_reg]
        else:
          if x_train[cont_reg,8]== 4:
            acum4=acum4+ y_train[cont_reg]
          else:
            if x_train[cont_reg,8]== 5:
              acum5=acum5+ y_train[cont_reg]
            else:
              if x_train[cont_reg,8]== 6:
                acum6=acum6+ y_train[cont_reg]
              else:
                if x_train[cont_reg,8]== 7:
                  acum7=acum7+ y_train[cont_reg]
                else:
                  if x_train[cont_reg,8]== 8:
                    acum8=acum8+ y_train[cont_reg]
                  else:
                    if x_train[cont_reg,8]== 9:
                      acum9=acum9+ y_train[cont_reg]
                    else:
                      if x_train[cont_reg,8]== 10:
                        acum10=acum10+ y_train[cont_reg]
                      else:
                        if x_train[cont_reg,8]== 11:
                          acum11=acum11+ y_train[cont_reg]
                        else:
                          if x_train[cont_reg,8]== 12:
                            acum12=acum12+ y_train[cont_reg]      
  else:
      pass
  cont_reg=cont_reg+1

x_g, y_g = [],[] 
for _ in range(1,13):
  x_g.append(_)
  if _ == 1:
    y_g.append(acum1)
  if _== 2:
    y_g.append(acum2)
  if _== 3:
    y_g.append(acum3)
  if _ == 4:
    y_g.append(acum4)
  if _== 5:
    y_g.append(acum5)
  if _== 6:
    y_g.append(acum6)
  if _ ==7:
    y_g.append(acum7)
  if _== 8:
    y_g.append(acum8)
  if _== 9:
    y_g.append(acum9)
  if _== 10:
    y_g.append(acum10)
  if _== 11:
    y_g.append(acum11)
  if _== 12:
    y_g.append(acum12) 

In [ ]:
y_g

In [ ]:
x_train

In [ ]:
y_train

In [ ]:
pd.DataFrame(x_train)

In [ ]:
pd.DataFrame(y_train)

In [239]:
x_train[:,8]

array([ 7,  4,  6,  5, 10,  9,  1,  7,  2,  8,  1,  8,  8,  3,  9,  6, 11,
        8,  1,  7,  1, 11,  9,  4,  6,  6,  3, 10,  2,  4,  4,  3,  5,  2,
        4, 11,  7,  3,  6,  1,  1,  1,  5,  7,  1,  3,  6,  1,  8,  6, 12,
        1,  5,  5,  8, 11,  6,  4, 11,  5,  2,  9,  4,  7,  5,  5,  3,  2,
        3,  5,  8,  5, 12, 10,  7, 12, 10, 12,  3,  7, 11, 12,  4,  6, 12,
        6,  7,  6,  6,  3, 10,  9,  2,  6,  1, 11,  1,  6,  5,  4, 12,  2,
        3, 10,  5,  7])

In [241]:
pd.DataFrame(x_train)

,0,1,2,3,4,5,6,7,8
0,0,0,0,1,0,0,0,2020,7
1,0,0,1,0,0,0,0,2020,4
2,0,0,0,0,0,0,1,2020,6
3,0,0,0,0,0,1,0,2019,5
4,0,0,0,0,0,1,0,2019,10
...,...,...,...,...,...,...,...,...,...
101,0,0,1,0,0,0,0,2019,2
102,0,0,0,0,0,1,0,2020,3
103,0,0,0,0,1,0,0,2019,10
104,0,0,0,0,0,1,0,2020,5


In [148]:
y[94]

array([144])

In [176]:
grupo.iloc[94,:]

Año           2020
Mes              2
Antiguedad       4
Cant_Bajas     144
Name: 94, dtype: int64

In [180]:
x_test[0,:]

array([   0,    0,    0,    1,    0,    0,    0, 2020,    2])

In [131]:
grupo.corr(method ='pearson')

,Año,Mes,Antiguedad,Cant_Bajas
Año,1.000000,-3.729741e-01,0.000000e+00,0.000419
Mes,-0.372974,1.000000e+00,-2.065386e-18,-0.032045
Antiguedad,0.000000,-2.065386e-18,1.000000e+00,0.701732
Cant_Bajas,0.000419,-3.204475e-02,7.017319e-01,1.000000
